In [1]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from time import sleep

from urllib3.exceptions import ProtocolError

In [2]:
df = pd.read_csv("wdrd.csv")
df.drop(columns=['Age', 'Date', 'DrugId', 'Sex', 'Sides', 'UsefulCount'], inplace=True)
df.columns = [d.lower() for d in df.columns]
df.columns

Index(['condition', 'drug', 'easeofuse', 'effectiveness', 'reviews',
       'satisfaction'],
      dtype='object')

In [3]:
#df.head()

In [4]:
with open('disease_symp_med_part.txt', 'r') as f:
    data = eval(f.read())
df2 = pd.DataFrame(data, columns=['disease', 'symptoms', 'medicine', 'part_of_body'])
df2['disease'] = [d[0] for d in df2['disease']]
df2.columns

Index(['disease', 'symptoms', 'medicine', 'part_of_body'], dtype='object')

In [5]:
s = set([_.lower() for _ in df['drug']])

In [6]:
s2 = set()
for _ in df2['medicine']:
    [s2.add(x.lower()) for x in _]

In [7]:
len(s2.difference(s)), len(s), len(s2)

(376, 7093, 577)

In [8]:
drugs = sorted(list(s2.difference(s)))

In [12]:
len(drugs)

376

In [10]:
from threading import Thread
from time import sleep

In [13]:
I = 231
d = {'drug':[], 'Effectiveness':[], 'Ease of Use':[], 'Satisfaction':[]}

In [ ]:
chrome_options = Options()
chrome_options.add_argument("--incognito")
#chrome_options.add_argument("--headless")
chrome_options.add_argument("--start-maximized")
browser = webdriver.Chrome("./chromedriver.exe", options=chrome_options)
file = "med_symp_eff.csv"
# with open(file, "w") as f:
#     f.write(','.join(['drug', 'effectiveness', 'ease of use', 'satisfaction']))
#     f.write("\n")
i = I
while(i<len(drugs)):
    try:
        drug = drugs[i]
        i += 1
        main_page = "https://www.webmd.com/drugs/2/search?type=drugs&query={}".format(drug)
        browser.get(main_page)
        
        try:
            browser.get(browser.find_element_by_class_name("exact-match").find_element_by_xpath("li/a").get_attribute("href"))
        except:
            pass
            #browser.find_element_by_tag_name("html").send_keys(Keys.ESCAPE)
            #browser.get(browser.find_element_by_class_name("exact-match").find_element_by_xpath("li/a").get_attribute("href"))
        try:
            #browser.find_element_by_class_name("drug-review-lowest").find_element_by_class_name("drug-review").click()
            #browser.find_element_by_class_name("drug-review").click()
            browser.get(browser.find_element_by_class_name("drug-review").get_attribute("href"))
        except:
            browser.find_element_by_tag_name("html").send_keys(Keys.ESCAPE)
            #browser.find_element_by_class_name("drug-review-lowest").find_element_by_class_name("drug-review").click()
            #browser.find_element_by_class_name("drug-review").click()
            #browser.get(browser.find_element_by_class_name("drug-review").get_attribute("href"))
            continue
            
        try:
            ratings = browser.find_element_by_id("ctnStars").find_elements_by_tag_name("div")
        except:
            I = i
            print("--->", drug)
            continue
            
#         ratings = ratings.find_elements_by_tag_name("div")
        
        if(len(ratings)<3):
            print("some error")
        else:
            m = []
            d['drug'].append(drug)
            for rating in ratings:
                r = rating.find_elements_by_tag_name("p")
                d[r[0].text].append(r[2].text[1:-1])
                m.append(r[2].text[1:-1])
            with open(file, "a") as f:
                f.write("{},{},{},{}\n".format(drug, m[0], m[1], m[2]))
        I = i
        if(i%3==0):
            browser.close()
            browser = webdriver.Chrome("./chromedriver.exe", options=chrome_options)
        print(i, drug)
    except ConnectionRefusedError as e:
        browser.close()
        print("->", drug)
        browser = webdriver.Chrome("./chromedriver.exe", options=chrome_options)
        continue
    except Exception as e:
        print(i, drug, e)
        i = I
        browser = webdriver.Chrome("./chromedriver.exe", options=chrome_options)

In [ ]:
len(pd.DataFrame(d))

86
86
86
86


In [ ]:
threads = []
x = len(drugs)/5
for i in range(5):
    s, e = int(i*x), int((i+1)*x)
    print(s, e)
    
    t = Thread(target=get_details, args=(drugs[s:e],))
    t.start()
    threads.append(t)
#d = {}
for t in threads:
    d_ = t.join()
    for k in d.keys():
        d[k].extend(d_[k])
        
print(len(d['Satisfaction']))

In [28]:
intersection = s2.intersection(s)
bubble_data = []

for i,row in df.iterrows():
    if row['drug'] in intersection:
        bubble_data.append([row['drug'],row['effectiveness'], row['easeofuse'],  row['satisfaction']])
        

In [ ]:
append = df[['drug','effectiveness','easeofuse','satisfaction']].groupby(by='drug').mean()
append.reset_index(inplace=True)
print(append)

In [47]:
final = pd.read_csv("med_symp_eff.csv")
#append = pd.DataFrame(bubble_data, columns=['drug','effectiveness','easeofuse','satisfaction'])
final = pd.concat([final, append])
final.reset_index(drop=True, inplace=True)

In [48]:
indexs = []
for i,row in final.iterrows():
    if(row['drug'] in s2):
        indexs.append(i)


In [50]:
final.loc[indexs].to_csv("med_symp_eff_final.csv")